In [457]:
import os
os.environ['MUJOCO_GL'] = 'egl'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

if 'notebooks' not in os.listdir(os.getcwd()):
    os.chdir('../') #changing directories so that output/gsplat_full etc. exists

from collision.utils import DummyCam, ImageDemoDataset, generate_camera, put_pose_into_mujoco, update_reconstruction_dict, get_normalized_function
from utils.mujoco_utils import compute_camera_extrinsic_matrix
from scene.cameras import Camera_Pose
from collision.chain_utils import build_chain_relation_map
from collision.network import SingleNetwork, HyperNetwork
from contextlib import redirect_stdout
from video_api import initialize_gaussians

import cv2
from gaussian_renderer import render
import sys 
import torch 
from PIL import Image
import numpy as np
import mujoco
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
from mujoco.usd import exporter
from transformers import CLIPProcessor, CLIPModel
from IPython.display import display, clear_output
from torchvision.transforms import transforms


import time
from pathlib import Path
from itertools import cycle

In [2]:
from torch.utils.data import Dataset
from typing import Dict
import json


class ImageDemoDataset(Dataset):
    def __init__(self, data_path: Path, background_color):
        self.cameras = sorted(data_path.glob("[0-9][0-9]"))
        self.camera_data = {}

        for camera_path in self.cameras:
            with (camera_path / "camera.json").open('r') as f:
                camera_info = json.load(f)
            self.camera_data[camera_path] = {
                "camera_info": camera_info,
                "image": sorted(camera_path.glob("[0-9][0-9][0-9][0-9].png")),
                "mask": sorted(camera_path.glob("seg_[0-9][0-9][0-9][0-9].png")),
                "depth": sorted(camera_path.glob("depth_[0-9][0-9][0-9][0-9].npy")),
            }

        self.action = np.load(data_path / "qpos.npy")

        self.preprocess = transforms.Compose([
            transforms.ToTensor(),
        ])

        self.background_tuple = background_color
    
    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        camera_list = []
        image_list = []
        mask_list = []
        depth_list = []

        for camera_data in self.camera_data.values():
            camera_list.append(camera_data['camera_info'])

            image = Image.open(camera_data['image'][index])
            seg = Image.open(camera_data['mask'][index])
            bg = Image.new("RGB", image.size, self.background_tuple)
            depth = np.load(camera_data['depth'][index]) * np.asarray(seg)

            result = Image.composite(image, bg, seg)

            image_list.append(self.preprocess(result).float())
            mask_list.append(self.preprocess(seg).float())
            depth_list.append(torch.as_tensor(depth).float())
        
        return {
            "action": self.action[index],
            "camera": camera_list,
            "image": image_list,
            "mask": mask_list,
            "depth": depth_list,
        }
    
    def __len__(self) -> int:
        return self.action.shape[0]

In [ ]:
sys.argv = ['']
gaussians, background_color, sample_cameras, kinematic_chain = initialize_gaussians(model_path='output/universal_robots_ur5e_robotiq')
kinematic_chain.to(device='cuda', dtype=torch.float)

In [4]:
# load mujoco
model = mujoco.MjModel.from_xml_path("mujoco_demo_control/universal_robots_ur5e_robotiq/scene.xml")
data = mujoco.MjData(model)

mujoco.mj_resetData(model, data)


def sample_collision_pose():
    pose = np.random.uniform(model.jnt_range[:, 0], model.jnt_range[:, 1])
    put_pose_into_mujoco(model, data, pose)
    return pose

In [5]:
background_tuple = (255, 255, 255)

demo_ds = ImageDemoDataset(Path("output/demonstration/universal_robots_ur5e_robotiq"), background_tuple)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(480, 480), dpi=1)
plt.imshow(demo_ds[120]['depth'][2])
plt.axis('off')
plt.subplots_adjust(0, 0, 1, 1)

In [983]:
mujoco.mj_resetData(model, data)

# set camera
dummy_cams = [
    DummyCam(0, -00.0, 2.5, lookat=[0,  0, 0]),
    DummyCam(0, -10.0, 2.5, lookat=[0,  0, 0]),
    DummyCam(0, 170, 2.5, lookat=[0,  0, 0]),
]
cams = [generate_camera(dummy_cam) for dummy_cam in dummy_cams]

renderer = mujoco.Renderer(model, 480, 480)
renderer.update_scene(data, camera=cams[0])

In [ ]:
mujoco.mj_step(model, data, 1000)

renderer.update_scene(data, camera=cams[0])
pixels = renderer.render()
image = Image.fromarray(pixels)
image

In [8]:
relation_map, chain = build_chain_relation_map("mujoco_menagerie/universal_robots_ur5e_robotiq/scene.xml")
sdf_model = HyperNetwork(6, relation_map)
state_dict = torch.load("output/universal_robots_ur5e_robotiq/sdf_net.ckpt", weights_only=True)
sdf_model.load_state_dict(state_dict)
for parameters in sdf_model.parameters():
    parameters.requires_grad_(False)
sdf_model.cuda()
del state_dict

In [9]:
norm_fun, unnorm_fun = get_normalized_function(*kinematic_chain.get_joint_limits())

init_params = [0.0] * 14
joint_angles = torch.nn.Parameter(
    torch.tensor(init_params, dtype=torch.float32, device='cuda')
)
optimizer = torch.optim.Adam([joint_angles], lr=0.01)


background_tuple = (255, 255, 255)
bg_color_t = torch.tensor(background_tuple).float().cuda() / 255.0

cam_list = []

for dummy_cam in dummy_cams:
    camera_extrinsic_matrix = compute_camera_extrinsic_matrix(dummy_cam)
    cam_list.append(
        Camera_Pose(torch.tensor(camera_extrinsic_matrix).clone().detach().float().cuda(), 0.78, 0.78, 480, 480, joint_pose=norm_fun(joint_angles), zero_init=True).cuda()
    )

In [109]:
def optimize(gaussians, joint_params, opt, background_color, gt_pkg: dict, norm_fun, max_iteration=200, callback=None):
    bg_color_t = torch.tensor(background_color).float().cuda() / 255.0

    camera_list, image_list, depth_list, gt_action = gt_pkg["camera"], gt_pkg["image"], gt_pkg["depth"], gt_pkg['action']

    image_list = [image.cuda() for image in image_list]
    depth_list = [depth.cuda() for depth in depth_list]

    gt_action = torch.tensor(gt_action[:14], dtype=torch.float32, device='cuda')

    def get_gs_camera(camera_info: dict):
        dummy_cam = DummyCam(camera_info['azimuth'], camera_info['elevation'], camera_info['distance'], lookat=camera_info['lookat'])
        camera_extrinsic_matrix = compute_camera_extrinsic_matrix(dummy_cam)
        return Camera_Pose(torch.tensor(camera_extrinsic_matrix).clone().detach().float().cuda(), 0.78, 0.78, 480, 480, joint_pose=norm_fun(joint_params), zero_init=True).cuda()
    
    camera_list = [get_gs_camera(camera_info) for camera_info in camera_list]

    camera_pkg = cycle(zip(camera_list, image_list, depth_list))
    
    tbar = trange(max_iteration, leave=False)
    for iteration in tbar:
        camera, gt_image, gt_depth = next(camera_pkg)

        camera.joint_pose = norm_fun(joint_params)
        output_pkg = render(camera, gaussians, bg_color_t)
        image_tensor = output_pkg['render']
        depth_tensor = output_pkg['depth']
        
        mask = gt_depth > 0
        Ll1_seg = F.l1_loss(image_tensor[:, mask], gt_image[:, mask])
        Ldepth_seg = F.l1_loss(depth_tensor[mask], gt_depth[mask])

        loss = Ll1_seg + Ldepth_seg

        loss = loss / len(camera_list)
        loss.backward()

        if iteration % len(camera_list) == len(camera_list)-1:
            if callback is not None:
                callback()

            opt.step()
            opt.zero_grad()

    return loss

In [11]:
topil = transforms.ToPILImage()

@torch.no_grad()
def render_image(gaussians, joint_params, background_color, gt_pkg: dict, norm_fun, idx=0):
    bg_color_t = torch.tensor(background_color).float().cuda() / 255.0

    camera, gt_image, gt_depth, gt_action = gt_pkg["camera"][idx], gt_pkg["image"][idx], gt_pkg["depth"][idx], gt_pkg['action']

    image_list = gt_image.cuda()
    depth_list = gt_depth.cuda()

    gt_action = torch.tensor(gt_action[:14], dtype=torch.float32, device='cuda')

    def get_gs_camera(camera_info: dict):
        dummy_cam = DummyCam(camera_info['azimuth'], camera_info['elevation'], camera_info['distance'], lookat=camera_info['lookat'])
        camera_extrinsic_matrix = compute_camera_extrinsic_matrix(dummy_cam)
        return Camera_Pose(torch.tensor(camera_extrinsic_matrix).clone().detach().float().cuda(), 0.78, 0.78, 480, 480, joint_pose=norm_fun(joint_params), zero_init=True).cuda()
    
    camera = get_gs_camera(camera)

    camera.joint_pose = norm_fun(joint_params)
    output_pkg = render(camera, gaussians, bg_color_t)
    image_tensor = output_pkg['render']
    depth_tensor = output_pkg['depth']

    clear_output(wait=True)
    display(topil(image_tensor))

In [ ]:
inverse_actions = []
# init joint angle to zero
with torch.no_grad():
    joint_angles[:] = 0
# optimize angle
for iteration, data_pkg in enumerate(tqdm(demo_ds)):
    optimize(gaussians, 
             joint_angles,
             optimizer, 
             background_tuple, 
             data_pkg,
             norm_fun,
             max_iteration=3*5,
             )
    inverse_actions.append(joint_angles.detach().cpu().numpy())

In [ ]:
for iteration, (data_pkg, joint_angle) in enumerate(zip(demo_ds, inverse_actions)):
    render_image(gaussians, 
                 torch.tensor(joint_angle, device='cuda'),
                 background_tuple, 
                 data_pkg,
                 norm_fun,
                 idx=0)
    time.sleep(0.05)

In [ ]:
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data, 1000)
init_collison = data.ncon
collision_list = []
for iteration, action in enumerate(tqdm(inverse_actions)):
    data.qpos[:6] = action[:6]
    mujoco.mj_step(model, data)
    mujoco.mj_collision(model, data)
    collision_list.append(data.ncon)
collision_list = np.asarray(collision_list) - init_collison
print('collision', np.mean(collision_list), np.mean(collision_list>0))

In [ ]:
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data, 1000)

exp = exporter.USDExporter(model=model)

last_gripper = 0
for iteration, action in enumerate(tqdm(inverse_actions)):
    data.ctrl[:6] = action[:6]
    if (action[6]+action[8]) > 0.6:
        data.ctrl[6] = 255

    renderer.update_scene(data, camera=cams[0])
    exp.update_scene(data=data)

    pixels = renderer.render()
    
    image = Image.fromarray(pixels)
    clear_output(wait=True)
    display(image)
    
    mujoco.mj_step(model, data, 10)
    mujoco.mj_collision(model, data)

    time.sleep(0.05)
exp.save_scene(filetype="usd")

In [196]:
target_action = torch.nn.Parameter(torch.tensor(inverse_actions, device='cuda', dtype=torch.float))
target_opt = torch.optim.Adam([target_action], lr=0.001)
init_joint_angle = torch.zeros_like(joint_angles)

In [ ]:
tbar = tqdm(demo_ds, leave=False)
for iteration, data_pkg in enumerate(tbar):
    def tv_loss():
        if iteration == 0:
            tv_loss = torch.nn.functional.l1_loss(target_action[0], target_action[1].detach())
        elif iteration == len(demo_ds) - 1:
            tv_loss = torch.nn.functional.l1_loss(target_action[iteration], target_action[iteration-1].detach())
        else:
            tv_loss = torch.nn.functional.l1_loss(target_action[iteration], target_action[iteration-1].detach()) + torch.nn.functional.l1_loss(target_action[iteration], target_action[iteration+1].detach())
            tv_loss = tv_loss / 2
                        
        tv_loss = tv_loss * 0.001
        tv_loss.backward()

    target_opt.zero_grad()

    loss = optimize(gaussians, 
                    target_action[iteration],
                    target_opt,
                    background_tuple, 
                    data_pkg,
                    norm_fun,
                    3*10,
                    callback=tv_loss
                    )
    
    tbar.set_postfix(
        {   
            "Iteration": iteration,
            "Loss": loss.item(),
        }
    )

    target_opt.step()

In [ ]:
with torch.no_grad():
    target_action_s0 = target_action.clone().detach()
target_action_s0

In [ ]:
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data, 1000)

exp = exporter.USDExporter(model=model)
last_gripper = 0
for iteration, action in enumerate(tqdm(target_action_s0.cpu().numpy())):
    data.ctrl[:6] = action[:6]
    if (action[6]+action[8]) > 0.6:
        data.ctrl[6] = 255

    renderer.update_scene(data, camera=cams[0])
    exp.update_scene(data=data)

    pixels = renderer.render()
    
    image = Image.fromarray(pixels)
    clear_output(wait=True)
    display(image)
    
    mujoco.mj_step(model, data, 10)
    mujoco.mj_collision(model, data)

    time.sleep(0.05)
exp.save_scene(filetype="usd")

In [ ]:
# use ee pose as constrict
with torch.no_grad():
    target_ee_pose = kinematic_chain.forward_kinematics(target_action_s0)['ur5e_0_wrist_3_link'].get_matrix().detach()

    plt.plot(target_action_s0[:, 6].detach().cpu().numpy())
ee_start = torch.nonzero(target_action_s0[:, 6] > 0.5)[0] - 20
ee_start

In [476]:
target_opt.zero_grad()
with torch.no_grad():
    target_action[:] = target_action_s0
target_opt = torch.optim.Adam([target_action], lr=0.00001)

In [ ]:
tbar = trange(10000)
for i in tbar:
    sdf_loss = torch.relu(sdf_model(target_action[:ee_start])[0]+0.10).mean()

    # ee pose
    ee_pose = kinematic_chain.forward_kinematics(target_action)['ur5e_0_wrist_3_link'].get_matrix()

    # xyz reconstriction for ee pose
    ee_tsl_loss = (ee_pose[ee_start:, :3, 3:]-target_ee_pose[ee_start:, :3, 3:]).norm(dim=-1).mean()
    ee_tv_loss = (ee_pose[1:, :3, 3:]-ee_pose[:-1, :3, 3:]).norm(dim=-1).mean()

    # TV loss
    tv_loss =  (target_action[1:]-target_action[:-1]).norm(dim=-1).mean()
    
    # movement
    move_loss = torch.nn.functional.mse_loss(target_action, target_action_s0)

    loss = 10.0 * sdf_loss + 10.0 * ee_tsl_loss + 1.0 * ee_tv_loss + 1.0 * tv_loss + 0.001 * move_loss
    # loss = loss * 100
    loss.backward()
    target_opt.step()

    tbar.set_postfix(
        {   
            "Loss": format(loss.item() ,"3.3e"),
            "SDF": format(sdf_loss.item(), ".4f"),
            "EET": format(ee_tsl_loss.item(), ".4f"),
            "ETV": format(ee_tv_loss.item(), ".4f"),
            "TV": format(tv_loss.item(), ".4f"),
            "MOVE": format(move_loss.item(), ".4f"),
        }
    )

In [ ]:
target_action.grad.max()

In [ ]:
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data, 1000)
init_collison = data.ncon
data.qpos[14:-1] = 0.9
collision_list = []
for iteration, action in enumerate(tqdm(target_action.detach().cpu().numpy())):
    data.qpos[:6] = action[:6]
    mujoco.mj_step(model, data)
    mujoco.mj_collision(model, data)
    collision_list.append(data.ncon)
collision_list = np.asarray(collision_list) - init_collison
print('collision', np.mean(collision_list), np.mean(collision_list>0))

In [ ]:
mujoco.mj_resetData(model, data)
mujoco.mj_step(model, data, 1000)
last_gripper = 0

exp = exporter.USDExporter(model=model)

for iteration, action in enumerate(tqdm(target_action.detach().cpu().numpy())):
    data.ctrl[:6] = action[:6]
    if action[6] > 0.6 and last_gripper == 0:
        last_gripper = 255

    data.ctrl[6] = last_gripper

    renderer.update_scene(data, camera=cams[0])
    exp.update_scene(data=data)

    pixels = renderer.render()
    
    image = Image.fromarray(pixels)
    clear_output(wait=True)
    display(image)
    
    mujoco.mj_step(model, data, 10)
    mujoco.mj_collision(model, data)

    time.sleep(0.05)
exp.save_scene(filetype="usd")

In [ ]:
put_pose_into_mujoco(model, data, joint_angles.detach().cpu().numpy())
print(data.ncon)
renderer.update_scene(data, camera=cams[0])
pixels = renderer.render()
image = Image.fromarray(pixels)
image

In [ ]:
model.jnt_range[:6, 0]

In [1153]:
pose_list = []

for i in range(100000):
    pose = np.random.uniform(
        np.array([np.pi/2, -np.pi, -np.pi, 0, 0, 0]), 
        np.array([np.pi/2,  np.pi,  np.pi, 0, 0, 0]))
    
    pose_list.append(pose[:6])
pose_list = np.asarray(pose_list)


In [ ]:
init_pose = pose_list[1000]
mujoco.mj_resetData(model, data)
data.qpos[:6] = pose_list[300]
data.qpos[-3:] = [0, 0, 10]
mujoco.mj_step(model, data)
print(data.ncon)

renderer.update_scene(data, camera=cams[1])
pixels = renderer.render()
image = Image.fromarray(pixels)
image

In [1155]:
xy_pose = pose_list[:, 1:3]

In [ ]:
sdf, s = sdf_model(torch.tensor(pose_list, dtype=torch.float32, device='cuda'))
sdf.max(), sdf.min()

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(xy_pose[:, 0], xy_pose[:, 1], c=sdf.detach().cpu().numpy()[:, 0], cmap='viridis')

In [1160]:
from scipy.interpolate import griddata

scale = 0.1

grid_x, grid_y = np.mgrid[-scale:scale:100j, -scale:scale:100j]  # 100j表示在0到10之间创建100个
grid_x -= 2.7
grid_y -= 1.5
grid_z = griddata((xy_pose[:, 0], xy_pose[:, 1]), sdf.detach().cpu().numpy()[:, 0], (grid_x, grid_y), method='cubic')

In [ ]:
plt.figure(figsize=(6, 6))
plt.contourf(grid_x, grid_y, grid_z, origin='lower', cmap='viridis')
# C = plt.contour(grid_x, grid_y, grid_z, colors ='black', linestyles='solid', linewidths=1)
plt.axis('off')
# plt.scatter(*xy_pose[34], c='r')
# plt.scatter(new_xy_p[:, 0], new_xy_p[:, 1], c='r')
plt.scatter(init_pose[1], init_pose[2], c='r')
plt.scatter(traj_stack[:, 1], traj_stack[:, 2], c='b')
plt.clabel(C, inline=True, fontsize=16)

In [ ]:
torch.nonzero((sdf.abs() > 0.1) & (sdf < 0.2))[1]

In [ ]:
sdf.shape

In [ ]:
i = 99686

print(xy_pose[i], sdf[i], pose_list[i])

In [ ]:
np.nonzero((xy_pose[:, 0] < -2.25) & (xy_pose[:, 0] > -3) & (xy_pose[:, 1] < -0.75) & (xy_pose[:, 1] > -1.5))

In [ ]:
pose_list[36]

In [1116]:
mujoco.mj_resetData(model, data)

# set camera
dummy_cams = [
    DummyCam(0, -00.0, 2.5, lookat=[0,  0, 0]),
    DummyCam(0, -10.0, 2.5, lookat=[0,  0, 0]),
    DummyCam(180, -20, 2, lookat=[0,  0, 0]),
]
cams = [generate_camera(dummy_cam) for dummy_cam in dummy_cams]

renderer = mujoco.Renderer(model, 480, 480)
renderer.update_scene(data, camera=cams[0])

In [ ]:
mujoco.mj_resetData(model, data)
data.qpos[:6] = traj_stack[4]
data.qpos[-3:] = [0, 0, 1]
mujoco.mj_step(model, data)
print(data.ncon)

exp = exporter.USDExporter(model=model)
exp.update_scene(data=data)
exp.save_scene(filetype="usd")
renderer.update_scene(data, camera=cams[2])
pixels = renderer.render()
image = Image.fromarray(pixels)
image


In [ ]:
traj_stack[0]

In [1164]:
init_pose = np.array([1.5008376, -2.727698, -1.5297332, -0.0697833, 0.06970214, -0.06995709])
init_pose_t = torch.nn.Parameter(torch.tensor(init_pose, dtype=torch.float32, device='cuda'))
optimizer = torch.optim.Adam([init_pose_t], lr=0.01)

In [ ]:
traj_stack = []
# SDF Loss 
for i in trange(1000):
    if (i % 2 == 0) and (i>0):
        traj_stack.append(init_pose_t.detach().cpu().numpy())

    optimizer.zero_grad()
    sdf, s = sdf_model(init_pose_t[None])
    sdf.backward()
    optimizer.step()
    print(i, sdf)
    
    if sdf < 0.5:
        break
print(len(traj_stack))

In [1130]:
traj_stack = np.asarray(traj_stack)

In [ ]:
sdf_model(init_pose_t[None])[0]

In [ ]:
traj_stack[0]-init_pose

In [ ]:
mujoco.mj_resetData(model, data)
data.qpos[:6] = init_pose
data.qpos[-3:] = [0, 0, 10]
mujoco.mj_step(model, data)
print(data.ncon)
renderer.update_scene(data, camera=cams[2])
pixels = renderer.render()
image = Image.fromarray(pixels)
image
# data.ncon

In [ ]:
traj_stack

In [ ]:
plt.figure(figsize=(6, 6))
plt.contourf(grid_x, grid_y, grid_z, origin='lower', cmap='viridis')
C = plt.contour(grid_x, grid_y, grid_z, colors ='black', linestyles='solid', linewidths=1)
plt.axis('off')
# plt.scatter(*xy_pose[75291], c='r')
plt.scatter(traj_stack[:, 1], traj_stack[:, 2], c='r')
plt.clabel(C, inline=True, fontsize=16)

In [ ]:
traj_stack

In [ ]:
plt_z = grid_z.T[10:90, 10:90] * 0.5
plt.figure(figsize=(6,6))

plt.contourf(plt_z, [-1, -0.75, -0.5, 0.0, 0.25, 0.5, 0.75, 1, 1.25], origin='lower', cmap='viridis')
C = plt.contour(plt_z, [-0.5, 0.0, 0.5], colors ='black', linestyles='solid', linewidths=5)
plt.axis('off')
plt.clabel(C, inline=True, fontsize=16)